# Inference pipeline for Chiayu 

This page shows how to run a trained model to get its prediction \
What is needed is:
* the model file in pt 
* the config.json for the initialization of the model architecture
* the vocab and merges file for the tokenizer -- we will then add our tokens inside so i dont have more files to deal with

In [1]:
from transformers import AutoTokenizer, AutoModelForPreTraining ,AutoConfig
import torch


In [2]:
model_dir = './colab_output/'
model_output = model_dir + 'stilted-snowball-14.pt'


SPECIAL_TOKENS  = { "bos_token": "<|BOS|>",
                    "eos_token": "<|EOS|>",
                    "unk_token": "<|UNK|>",                    
                    "pad_token": "<|PAD|>",
                    "sep_token": "<|SEP|>"}

tokenizer = AutoTokenizer.from_pretrained(model_dir) 
tokenizer.add_special_tokens(SPECIAL_TOKENS)

model_config = AutoConfig.from_pretrained(model_dir+'config.json', 
                                    bos_token_id=tokenizer.bos_token_id,
                                    eos_token_id=tokenizer.eos_token_id,
                                    sep_token_id=tokenizer.sep_token_id,
                                    pad_token_id=tokenizer.pad_token_id,
                                    output_hidden_states=False)

model = AutoModelForPreTraining.from_pretrained(model_output, config=model_config)



In [3]:
#device = 'cpu' 
device = 'cuda'

rating = 'boring, not funny, not funny, horrible'

def gen_suggestions(rating,model):
    prompt = SPECIAL_TOKENS['bos_token'] + rating + SPECIAL_TOKENS['sep_token']
    generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0).to(device)
    model.to(device)
    model.eval();

      # Top-p (nucleus) text generation (10 samples):
    sample_outputs = model.generate(generated, 
                                      do_sample=True,   
                                      min_length=50, 
                                      max_length=256,
                                      top_k=30,                                 
                                      top_p=0.7,        
                                      temperature=0.9,
                                      repetition_penalty=2.0,
                                      num_return_sequences=3
                                      )

    for i, sample_output in enumerate(sample_outputs):
            text = tokenizer.decode(sample_output, skip_special_tokens=True)   

            print("{}: {}\n\n".format(i+1,  text[len(rating):]))

gen_suggestions('boring, not funny, not funny, horrible',model)

1: I ordered the Chicken and Bologna.  The chicken was bland...but I had to order it again after a week of eating there for lunch so that we could have some decent food at home! We were very disappointed with how much they put on our table when ordering from us as well!!
We got seated right away but didn't realize why until about 15 minutes later which took forever (it's an hour). There wasn'nt even any sauce or dressing in either dish except salt..the meat tasted like sausage instead....so salty!!! Our waitress came out feeling bad because she knew what would happen if this happened - my boyfriend just walked by me looking miserable without having his meal prepared properly.....he never left her alone during dinner time, he only wanted one person working around him! He is extremely unprofessional here and will NEVER leave you anywhere else!!!!


2: This place has been my go-to spot for a while now. They have some of the best breakfast in town and I highly recommend it to anyone lookin